In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dask DataFrame

Материалы: 
* Макрушин С.В. Лекция 13: Dask DataFrame
* https://docs.dask.org/en/latest/dataframe.html
* JESSE C. DANIEL. Data Science with Python and Dask. 

---
[data Gdrive](https://drive.google.com/drive/u/1/folders/1p78TQuvPlHZtJa2p_l3PEZQDNmvph9go)
---


## Задачи для совместного разбора

In [ ]:
from pathlib import Path
import dask.dataframe as dd

In [ ]:
datafolder = Path('/content/drive/MyDrive/FinUniver/TOBD_2022/data/accounts')
print([i.name for i in datafolder.glob('*')])

['accounts.0.csv', 'accounts.1.csv', 'accounts.2.csv']


In [ ]:
df = dd.read_csv(datafolder / '*.csv',dtype={'amount': 'float64'})
df.head()

,id,names,amount
0,39,Victor,205.0
1,289,Hannah,87.0
2,234,Victor,1820.0
3,155,Ray,-600.0
4,266,Ray,572.0


1. Считать данные из файлов в каталоге `accounts`. Содержат ли какие-либо из столбцов пропуски?

In [ ]:
df.isna().sum().compute()

id        0
names     0
amount    3
dtype: int64

2. Подсчитать количество раз, которое то или иное имя встретилось в выборке. Вывести самое часто встречающееся имя.

In [ ]:
df.names.value_counts().compute()

Norbert     188147
Alice       185892
George      183249
Tim         162432
Bob         157065
Michael     148372
Ingrid      132067
Oliver      121907
Quinn       118071
Wendy       110657
Charlie     109236
Ursula      108745
Hannah      108632
Ray         108610
Sarah       104781
Victor      102656
Frank        99984
Laura        97216
Jerry        96378
Xavier       94445
Edith        89991
Zelda        89047
Kevin        84784
Dan          73293
Patricia     62881
Yvonne       61462
Name: names, dtype: int64

In [ ]:
df.names.value_counts().compute()[0:1]

Norbert    188147
Name: names, dtype: int64

3. Создать новую колонку, которая является результатом от деления значения `amount` нацело на 100, если `amount` > 100, и нулём в противном случае.

In [ ]:
df['new'] = df['amount'].map(lambda x: x//100 if x > 100 else 0)
df.head(5)

,id,names,amount,new
0,39,Victor,205.0,2.0
1,289,Hannah,87.0,0.0
2,234,Victor,1820.0,18.0
3,155,Ray,-600.0,0.0
4,266,Ray,572.0,5.0


4. В accounts.*.csv найти id, для которого в столбце amount встречается наибольшее количество значений, кратных трем. 



In [ ]:
df.where(df['amount']%3==0)['id'].value_counts().compute()

137.0    16283
448.0    10812
290.0    10640
228.0    10507
173.0    10402
         ...  
169.0        7
428.0        4
334.0        2
238.0        2
128.0        1
Name: id, Length: 499, dtype: int64

5. В accounts.*.csv найти id, для которого сумма положительных значений в столбце amount наибольшая. 

In [ ]:
df_pos = df[df["amount"] >= 0]

In [ ]:
df_pos.groupby('id')["amount"].sum().idxmax().compute()

137

6. В accounts.*.csv найти id, для которого в столбце amount встречается наибольшее количество значений, кратных 5 и не кратных 10. 

In [ ]:
df.where(df['amount'] % 5==0).where(df['amount']%10!=0)['id'].value_counts().compute()

137.0    4883
243.0    4857
290.0    3183
228.0    3158
173.0    3095
         ... 
238.0       3
169.0       3
428.0       3
334.0       2
413.0       1
Name: id, Length: 496, dtype: int64

7. В accounts.*.csv найти id, для которого в столбце amount встречается наибольшее количество значений между 1000 и 1500.

## Лабораторная работа 13

In [1]:
import dask.dataframe as dd
import pandas as pd
import dask.bag as db
import json
import dask

1. В архиве `recipes_full.zip` находятся файлы, содержащие информацию об рецептах блюд. Загрузите данные из файлов этого архива в виде `dd.DataFrame` с названием `recipes`. Укажите, что в столбце `submitted` содержатся даты.

In [2]:
recipes = dd.read_csv("./recipes_full/recipes_full_*.csv", 
                      parse_dates = ["submitted"], dtype={'minutes': 'float64','n_steps': 'float64'})

In [3]:
recipes.head()

,id,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,683970,vant ivoire mickies nothing,33.0,803776,2019-08-22,4.0,pat and gina neely and their family own and op...,9
1,1089012,kremsils mariposa baccala cookies class borage...,23.0,51579,2013-03-02,1.0,"a light, tasty and easy to put together chicke...",5
2,1428572,tania lander,0.0,68884,1980-11-09,1.0,a delicious melt in your mouth appetizer. for ...,5
3,1400250,heloise milli asher doogh zojirushi,24.0,678862,2018-04-29,3.0,delicious cream cheese and peach filled cresce...,1
4,387709,nutty chocolate chunk cookies,47.0,489552,2009-08-31,8.0,everyone loves these buttery cookies chock ful...,10


2. Выведите метаинформацию о таблице: `npartitions` и типы столбцов.

In [4]:
recipes.npartitions

8

In [5]:
recipes.dtypes

id                         int64
name                      object
minutes                  float64
contributor_id             int64
submitted         datetime64[ns]
n_steps                  float64
description               object
n_ingredients              int64
dtype: object

3. Выведите на экран 5 первых строк таблицы. Выведите на экран 5 последних строк таблицы. В случае сообщения об ошибки объясните причину и исправьте ошибку.

In [6]:
recipes.head(n=5)

,id,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,683970,vant ivoire mickies nothing,33.0,803776,2019-08-22,4.0,pat and gina neely and their family own and op...,9
1,1089012,kremsils mariposa baccala cookies class borage...,23.0,51579,2013-03-02,1.0,"a light, tasty and easy to put together chicke...",5
2,1428572,tania lander,0.0,68884,1980-11-09,1.0,a delicious melt in your mouth appetizer. for ...,5
3,1400250,heloise milli asher doogh zojirushi,24.0,678862,2018-04-29,3.0,delicious cream cheese and peach filled cresce...,1
4,387709,nutty chocolate chunk cookies,47.0,489552,2009-08-31,8.0,everyone loves these buttery cookies chock ful...,10


In [7]:
recipes.tail(n=5)

,id,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
278949,1029131,tuti waffle snackies steakhouse,19.0,171345,1973-10-18,4.0,"according to a providence journal article, ama...",4
278950,1700703,noelias cheats crocante fleisch zitumbuwa,1.0,30228,2007-07-01,6.0,if possible sauté the onions and garlic in abo...,1
278951,1910650,rubbed restuffed pelmeni bedouin flavourful,60.0,591905,2009-09-26,3.0,another great recipe to add to the growing swe...,2
278952,713836,stems polpettine peezi,NaN,357389,2003-09-30,4.0,adapted from top secret recipes. love this!,9
278953,660699,clementines,64.0,29196,1973-06-03,6.0,this would make a great start to your holiday ...,8


4. Посчитайте, сколько строк содержит каждый из блоков

In [8]:
recipes.map_partitions(len).compute()

0    278955
1    278955
2    278955
3    278955
4    278955
5    278954
6    278954
7    278954
dtype: int64

5. Найдите максимум в столбце `n_steps`. Визуализируйте граф вычислений для этой задачи. Прокомментируйте логику работы `dask` в этом случае.

In [18]:
my_calculation = recipes["n_steps"].max()
my_calculation.compute()

145.0

6. Посчитайте количество отзывов с группировкой по месяцам добавления отзыва в базу.

In [19]:
data = recipes.groupby(recipes['submitted'].dt.month).count().compute()
task6_result = pd.DataFrame(data["submitted"].rename("recipes_count"), data.index.rename("month"))
task6_result

,recipes_count
month,
1,193363
2,173834
3,192389
4,186049
5,192487
6,184205
7,189337
8,187276
9,181081


7. Считайте файлы из архива `reviews_full.zip` (__ЛР12__) в виде `dask.bag`. Пользуясь результатом лабораторной работы 12, рассчитайте среднее значение оценок отзывов с группировкой по месяцам. После завершения всех вычислений преобразуйте результат к `pd.Series`.

In [20]:
import re
from datetime import datetime
def my_loader(data) -> dict:

    json_str, path = data
    file_number = re.findall('reviews_([0-9]).json', path)
    
    if file_number is None or len(file_number) != 1:
        raise ValueError("Не могу извлечь номер файла")
    
    json_obj = json.loads(json_str)
    json_obj["rating"] = int(file_number[0])
    json_obj["date"] = datetime.strptime(json_obj["date"], '%Y-%m-%d')
    return json_obj

In [22]:
reviews_bag = db.read_text(urlpath="./reviews_full/reviews_full/reviews_*.json", include_path=True).map(my_loader)
reviews_bag.take(5)

({'user_id': 452355,
  'recipe_id': 292657,
  'date': datetime.datetime(2016, 5, 8, 0, 0),
  'review': 'WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.',
  'rating': 0},
 {'user_id': 329304,
  'recipe_id': 433404,
  'date': datetime.datetime(2006, 6, 14, 0, 0),
  'review': 'This was good but the dressing needed something and I found it to be a little too sweet, next time I will experiment with some garlic and herbs and reduce the sugar slightly, thanks for sharing kcdlong!...Kitten',
  'rating': 0},
 {'user_id': 227932,
  'recipe_id': 2008187,
  'date': datetime.datetime(1985, 11, 19, 0, 0),
  'review': 'Very good,it was a hit for my family. I used 6 cloves of garlic and had 1 lb beef and  Johnsonville sausage,1/2 lb hot and  1/2 lb honey garlic( which I wanted to use). That was a pe

In [23]:
reviews_df = reviews_bag.to_dataframe()
reviews_df

,user_id,recipe_id,date,review,rating
npartitions=6,,,,,
,int64,int64,datetime64[ns],object,int64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [24]:
task7_result = reviews_df.groupby(reviews_df['date'].dt.month)["rating"].mean().compute()
task7_result

date
1     4.408553
2     4.407135
3     4.406911
4     4.412259
5     4.415401
6     4.418419
7     4.411717
8     4.410822
9     4.410661
10    4.411452
11    4.408829
12    4.406908
Name: rating, dtype: float64

8. Пользуясь результатами решения задач 6 и 7, создайте `pd.DataFrame`, содержащий два столбца: `mean_rating`, `recipes_count`

In [25]:
task6_result["mean_rating"] = task7_result
task6_result

,recipes_count,mean_rating
month,,
1,193363,4.408553
2,173834,4.407135
3,192389,4.406911
4,186049,4.412259
5,192487,4.415401
6,184205,4.418419
7,189337,4.411717
8,187276,4.410822
9,181081,4.410661
